# 1. Import packages

In [1]:
import copy
import os
import pandas as pd

from collections import Counter

# 2. Read a CSV

## 2.1. Change to output directory

In [2]:
print(os.getcwd())
input_path_s = "/input/"
input_path_s = os.getcwd() + input_path_s
output_path_s = "/output/"
output_path_s = os.getcwd() + output_path_s
os.chdir(output_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain
/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task17_target_domain/output


In [3]:
bright_data_14_df = pd.read_csv("domains_bright_data_14col.csv", index_col=0)

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
bright_data_14_df.shape

(1075319, 14)

In [5]:
bright_data_14_df.iloc[50:60]

,country,asn,initial_domain_filename,target_domain,initial_domain_tld_plus_one,target_domain_tld_plus_one,tld_plus_one_first_party,as_name_first_party,as_number_first_party,org_id_first_party,org_name_first_party,first_party,is_target_domain_tracker,is_target_domain_google
50,hr,61094,20220804-HR-61094-httpswww.tportal.hr-DNS_Requ...,www.tportal.hr,tportal.hr,tportal.hr,True,NaN,NaN,NaN,NaN,True,False,False
51,hr,61094,20220804-HR-61094-httpswww.poslovni.hr-DNS_Req...,hr.hit.gemius.pl,poslovni.hr,gemius.pl,NaN,True,True,True,True,True,True,False
52,hr,61094,20220804-HR-61094-httpswww.net.hr-DNS_Requests...,s.amazon-adsystem.com,net.hr,amazon-adsystem.com,NaN,True,True,True,True,True,False,False
53,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,www.youtube.com,gloria.hr,youtube.com,NaN,True,True,True,True,True,True,False
54,hr,61094,20220804-HR-61094-httpswww.index.hr-DNS_Reques...,script.dotmetrics.net,index.hr,dotmetrics.net,NaN,NaN,NaN,NaN,NaN,NaN,True,False
55,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,connect.facebook.net,vecernji.hr,facebook.net,NaN,True,True,True,True,True,True,False
56,hr,61094,20220804-HR-61094-httpswww.vecernji.hr-DNS_Req...,cdnjs.cloudflare.com,vecernji.hr,cloudflare.com,NaN,True,True,True,True,True,True,False
57,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-beacon.s-onetag.com,jutarnji.hr,s-onetag.com,NaN,NaN,NaN,NaN,NaN,NaN,True,False
58,hr,61094,20220804-HR-61094-httpswww.gloria.hr-DNS_Reque...,apps.jutarnji.hr,gloria.hr,jutarnji.hr,NaN,True,True,True,True,True,True,False
59,hr,61094,20220804-HR-61094-httpswww.jutarnji.hr-DNS_Req...,signal-metrics-collector-beta.s-onetag.com,jutarnji.hr,s-onetag.com,NaN,NaN,NaN,NaN,NaN,NaN,True,False


In [6]:
bright_data_14_df.columns

Index(['country', 'asn', 'initial_domain_filename', 'target_domain',
       'initial_domain_tld_plus_one', 'target_domain_tld_plus_one',
       'tld_plus_one_first_party', 'as_name_first_party',
       'as_number_first_party', 'org_id_first_party', 'org_name_first_party',
       'first_party', 'is_target_domain_tracker', 'is_target_domain_google'],
      dtype='object')

# 3. Add sanity column

## 3.1 first_party, is_target_domain_tracker

In [7]:
print(bright_data_14_df.first_party.value_counts())
print()
print(f"{sum(pd.isna(bright_data_14_df.first_party))} NaNs.")
print(f"{round(float(bright_data_14_df.first_party.value_counts()) / len(bright_data_14_df) * 100.0, 5)}% are first party.")

True    843909
Name: first_party, dtype: int64

231410 NaNs.
78.47987% are first party.


In [8]:
print(bright_data_14_df.is_target_domain_tracker.value_counts())
print()
print(f"{sum(pd.isna(bright_data_14_df.is_target_domain_tracker))} NaNs.")
print(f"{round(list(bright_data_14_df.is_target_domain_tracker).count(True) / len(bright_data_14_df) * 100.0, 5)}% are tracker.")

True     712815
False    362504
Name: is_target_domain_tracker, dtype: int64

0 NaNs.
66.2887% are tracker.


## 3.2. not_first_party_target_domain

In [9]:
not_first_party_target_domain_l = []

for i in range(len(bright_data_14_df)):
    if bright_data_14_df["first_party"][i] is True:
        not_first_party_target_domain_l.append("first_party")
    else:
        not_first_party_target_domain_l.append(bright_data_14_df["target_domain"][i])

print(len(not_first_party_target_domain_l) == len(bright_data_14_df))
print(not_first_party_target_domain_l.count('first_party'))
print(f"{round(100 - not_first_party_target_domain_l.count('first_party') / len(bright_data_14_df) * 100, 5)}% are not a first party.")

True
843909
21.52013% are not a first party.


## 3.3. not_tracker_target_domain

In [10]:
not_tracker_target_domain_l = []

for i in range(len(bright_data_14_df)):
    if bright_data_14_df["is_target_domain_tracker"][i]:
        not_tracker_target_domain_l.append("tracker")
    else:
        not_tracker_target_domain_l.append(bright_data_14_df["target_domain"][i])

print(len(not_tracker_target_domain_l) == len(bright_data_14_df))
print(not_tracker_target_domain_l.count('tracker'))
print(f"{round(100 - not_tracker_target_domain_l.count('tracker') / len(bright_data_14_df) * 100, 5)}% are not a tracker.")

True
712815
33.7113% are not a tracker.


## 3.4. Not Google ASN

In [11]:
print(Counter(bright_data_14_df["asn"])["google"])
print(f'{round(Counter(bright_data_14_df["asn"])["google"] / len(bright_data_14_df) * 100.00, 5)}%')

637
0.05924%


## 3.5. not_first_party_and_not_tracker_l

In [22]:
not_first_party_and_not_tracker_l = []

for i in range(len(bright_data_14_df)):
    if bright_data_14_df["asn"][i] != "google" and bright_data_14_df["first_party"][i] is not True and not bright_data_14_df["is_target_domain_tracker"][i]:
        not_first_party_and_not_tracker_l.append(bright_data_14_df["target_domain_tld_plus_one"][i])
    else:
        not_first_party_and_not_tracker_l.append(None)
        
print(len(not_first_party_and_not_tracker_l) == len(bright_data_14_df))
print(not_first_party_and_not_tracker_l.count(None))
print(f"{100 - round(not_first_party_and_not_tracker_l.count(None) / len(bright_data_14_df) * 100, 5)}% are neither a first party nor a tracker.")

True
1015394
5.572760000000002% are neither a first party nor a tracker.


In [13]:
# Counter(not_first_party_and_not_tracker_l)

# 4. Add column

In [24]:
bright_data_14_df.shape

(1075319, 14)

In [25]:
bright_data_15_df = copy.deepcopy(bright_data_14_df)

In [26]:
bright_data_15_df["not_first_party_and_not_tracker"] = not_first_party_and_not_tracker_l

In [27]:
bright_data_15_df.shape

(1075319, 15)

In [30]:
bright_data_15_df.to_csv("domains_bright_data_15col.csv")

# 5. value_counts()

In [35]:
for (column_name, column_data) in bright_data_15_df.iteritems():
    print('Column Name : ', column_name)
    print(bright_data_15_df[column_name].value_counts())
    print()

Column Name :  country
cz    213228
pl    188829
es    122239
it    100208
bg     67055
nl     57309
de     46599
fr     43535
ro     31315
se     30797
sk     29239
hu     25544
ie     20226
hr     20017
at     19572
dk     15386
gr     14118
fi     13254
be      9013
pt      7836
Name: country, dtype: int64

Column Name :  asn
8220      3012
16509     2863
6830      2843
9009      2794
202422    2793
          ... 
20773      308
42652      305
25291      305
15699      116
42401       44
Name: asn, Length: 1413, dtype: int64

Column Name :  initial_domain_filename
20220805-ES-766-httpswww.elmundo.es-DNS_Requests.txt         163
20220806-ES-34285-httpswww.telecinco.es-DNS_Requests.txt     150
20220805-ES-44278-httpswww.telecinco.es-DNS_Requests.txt     146
20220805-ES-199952-httpswww.telecinco.es-DNS_Requests.txt    145
20220805-ES-44431-httpswww.telecinco.es-DNS_Requests.txt     143
                                                            ... 
20220806-NL-58061-httpswww.plejada.p

In [37]:
bright_data_15_df.shape

(1075319, 15)

In [41]:
print(bright_data_15_df.shape[0] - 176550)
print(bright_data_15_df.shape[0] - 789841)
print(bright_data_15_df.shape[0] - 793573)

898769
285478
281746


In [43]:
712815 + 362504 == bright_data_15_df.shape[0]

True

In [44]:
Counter(bright_data_15_df.not_first_party_and_not_tracker)

Counter({None: 1015394,
         'rbxcdn.com': 2791,
         'unizg.hr': 19,
         'erato.hr': 19,
         'midas-network.com': 57,
         'openx.net': 435,
         'npttech.com': 43,
         'adpushup.com': 46,
         'creativecdn.com': 1422,
         'zagreb.info': 19,
         '4dex.io': 298,
         'sysbee.net': 19,
         'rubiconproject.com': 1789,
         'linker.hr': 95,
         'adrecover.com': 72,
         'twimg.com': 2830,
         'adsafeprotected.com': 342,
         '7dnevno.hr': 19,
         'googletagservices.com': 1710,
         '234doo.com': 38,
         'moatads.com': 211,
         'smartadserver.com': 733,
         'google.hr': 76,
         'bootstrapcdn.com': 250,
         'polyfill.io': 354,
         'w-x.co': 186,
         'adnetwork.agency': 153,
         'amazon-adsystem.com': 1991,
         'mookie1.com': 279,
         'adnxs.com': 1274,
         'webhosting-wmd.hr': 19,
         'cxpublic.com': 19,
         'yieldlove.com': 19,
         'arko